In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
np.random.seed(10)  # Generate the data.
xTrain = np.array([-1.0, -0.8, -0.2, 0.5, 0.7])
yTrain = -(xTrain ** 2) + 10.0
yTrain[3] -= 0.2
xTest = np.array([-1.1, -0.6, 0.1, 0.3])
yTest = -(xTest ** 2) + 10.0

In [5]:
import numpy as np

class HigherOrderRegression:
    def __init__(self, polynomialDegree, regularization=None):
        self.polynomialDegree = int(polynomialDegree)
        self.regularization = regularization
        self.weights = np.zeros(self.polynomialDegree, dtype=float)
        self.bias = 0.0

    def forward(self, x):
        x = np.asarray(x, dtype=float).reshape(-1)          # (N,)
        y = np.full_like(x, fill_value=self.bias, dtype=float)
        for i in range(self.polynomialDegree):
            y += self.weights[i] * (x ** (i + 1))
        return y                                            # (N,)

    # alias útil
    def predict(self, x):
        return self.forward(x)

    def costFunction(self, x, y):
        x = np.asarray(x, dtype=float).reshape(-1)
        y = np.asarray(y, dtype=float).reshape(-1)
        err = self.forward(x) - y
        cost = np.mean(err ** 2)
        if self.regularization is not None:
            cost += self.regularization * (np.sum(self.weights ** 2) + self.bias ** 2)
        return cost

    def gradient(self, x, y):
        x = np.asarray(x, dtype=float).reshape(-1)
        y = np.asarray(y, dtype=float).reshape(-1)
        err = self.forward(x) - y                           # (N,)
        grad_b = 2 * np.mean(err)
        grad_w = np.zeros(self.polynomialDegree, dtype=float)
        for i in range(self.polynomialDegree):
            grad_w[i] = 2 * np.mean(err * (x ** (i + 1)))
            if self.regularization is not None:
                grad_w[i] += 2 * self.regularization * self.weights[i]
        # Obs.: não regularizamos o bias (como no seu comentário)
        return grad_w, grad_b

    def train(self, epochs, lr, xTrain, yTrain, xTest=None, yTest=None, verbose=100):
        for epoch in range(epochs):
            costTrain = self.costFunction(xTrain, yTrain)
            costTest = self.costFunction(xTest, yTest) if (xTest is not None and yTest is not None) else np.nan

            grad_w, grad_b = self.gradient(xTrain, yTrain)
            self.weights -= lr * grad_w
            self.bias    -= lr * grad_b

            if verbose and (epoch % verbose == 0 or epoch == epochs - 1):
                print(f"Epoch: {epoch}/{epochs}\tTraining cost = {costTrain:.2e}\tValidation cost = {costTest:.2e}")
        return self


In [6]:
polynomialDegree = 9
lr = 1e-1
epochs = 1000
regularization = 1e-2

model = HigherOrderRegression(polynomialDegree)
# model = HigherOrderRegression(polynomialDegree, regularization) # activate the regularization
model.train(epochs, lr, xTrain, yTrain, xTest, yTest)

Epoch: 0/1000	Training cost = 8.99e+01	Validation cost = 9.21e+01
Epoch: 100/1000	Training cost = 1.10e-01	Validation cost = 5.65e-01
Epoch: 200/1000	Training cost = 3.66e-02	Validation cost = 2.21e-01
Epoch: 300/1000	Training cost = 1.38e-02	Validation cost = 7.67e-02
Epoch: 400/1000	Training cost = 6.19e-03	Validation cost = 3.38e-02
Epoch: 500/1000	Training cost = 3.46e-03	Validation cost = 2.35e-02
Epoch: 600/1000	Training cost = 2.34e-03	Validation cost = 2.31e-02
Epoch: 700/1000	Training cost = 1.78e-03	Validation cost = 2.53e-02
Epoch: 800/1000	Training cost = 1.44e-03	Validation cost = 2.80e-02
Epoch: 900/1000	Training cost = 1.18e-03	Validation cost = 3.06e-02
Epoch: 999/1000	Training cost = 9.83e-04	Validation cost = 3.30e-02
